# Carregando os dados do DuckDB

O objetivo dessa etapa é realizar a carga nos dados no banco duckdb.

## Etapas realizadas neste código:
1. Importação das bibliotecas necessárias (duckdb, pandas, time, os, datetime).
2. Definição de uma função de log para registrar mensagens com timestamp.
3. Mapeamento dos arquivos de uma pasta para um dicionário com nomes e caminhos completos.
4. Conexão ao banco de dados DuckDB.
5. Criação/verificação do schema 'raw' no banco de dados.
6. Inserção dos arquivos mapeados como tabelas no schema 'raw' do DuckDB, utilizando detecção automática de formato.
7. Criação das demais zonas de dados


## 1- Import e criação das funções

In [1]:
!pip install duckdb

In [2]:
import duckdb
import pandas as pd
import time
import os
from datetime import datetime

# Função utilitária para log com horário
def log(msg):
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}")

In [3]:
def mapear_arquivos_pasta(pasta):
    """
    Mapeia todos os arquivos da pasta e retorna um dicionário
    com o nome do arquivo como chave e o caminho completo como valor.
    """
    arquivos_dict = {}
    for root, dirs, files in os.walk(pasta):
        for file in files:
            caminho_completo = os.path.join(root, file)
            arquivos_dict[file] = caminho_completo
    return arquivos_dict

In [4]:
def inserir_arquivos_no_duckdb(arquivos_dict, con):
    """
    Para cada arquivo no dicionário, checa se a tabela existe.
    Se não existir, insere usando read_csv_auto ou read_parquet.
    Tenta diferentes encodings para arquivos CSV caso o padrão UTF-8 falhe.
    """
    
    # Garante que o schema 'raw' existe antes de inserir os dados
    try:
        con.execute("CREATE SCHEMA IF NOT EXISTS raw;")
        print("📁 Schema criado/verificado: raw")
    except Exception as e:
        log(f"Erro ao criar/verificar schema 'raw': {e}")
        raise
    
    # Lista de encodings para tentar (em ordem de prioridade) - usando nomes suportados pelo DuckDB
    # latin-1 é equivalente a ISO-8859-1
    encodings = ['utf-8', 'latin-1']
    
    # Adiciona os dados no schema 'raw'
    for nome_arquivo, caminho_arquivo in arquivos_dict.items():
        nome_tabela = os.path.splitext(nome_arquivo)[0]
        tabela_com_schema = f"raw.{nome_tabela}"
        
        # Checa se a tabela já existe no schema 'raw'
        try:
            existe = con.execute(
                f"SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'raw' AND table_name = '{nome_tabela}'"
            ).fetchone()[0]
            if existe:
                log(f"Tabela '{tabela_com_schema}' já existe. Pulando inserção.")
                continue
        except Exception as e:
            log(f"Erro ao verificar existência da tabela '{tabela_com_schema}': {e}")
        
        # Detecta tipo de arquivo e monta comando
        if caminho_arquivo.endswith(".parquet"):
            comando = f"""
                CREATE TABLE {tabela_com_schema} AS SELECT * FROM read_parquet('{caminho_arquivo}')
            """
            log(f"Inserindo '{caminho_arquivo}' na tabela '{tabela_com_schema}'...")
            try:
                con.execute(comando)
                log(f"Tabela '{tabela_com_schema}' criada e dados inseridos com sucesso.")
            except Exception as e:
                log(f"Erro ao inserir '{caminho_arquivo}': {e}")
                continue
                
        elif caminho_arquivo.endswith(".csv") or caminho_arquivo.endswith(".CSV"):
            # Tenta diferentes encodings
            sucesso = False
            # Converte para caminho absoluto e normaliza
            caminho_absoluto = os.path.abspath(caminho_arquivo)
            caminho_normalizado = caminho_absoluto.replace('\\', '/')
            
            for encoding in encodings:
                try:
                    # Usa read_csv_auto com parâmetros específicos para CSV com ponto e vírgula
                    comando = f"""
                        CREATE TABLE {tabela_com_schema} AS 
                        SELECT * FROM read_csv_auto('{caminho_normalizado}', 
                            encoding='{encoding}', 
                            delim=';',
                            header=true,
                            ignore_errors=false)
                    """
                    log(f"Inserindo '{caminho_arquivo}' na tabela '{tabela_com_schema}' (tentando encoding: {encoding})...")
                    con.execute(comando)
                    log(f"✅ Tabela '{tabela_com_schema}' criada e dados inseridos com sucesso (encoding: {encoding}).")
                    sucesso = True
                    break
                except Exception as e:
                    if encoding == encodings[-1]:  # Último encoding da lista
                        log(f"❌ Erro ao inserir '{caminho_arquivo}' com todos os encodings tentados. Último erro: {e}")
                    else:
                        log(f"⚠️ Encoding {encoding} falhou, tentando próximo...")
                    continue
            
            if not sucesso:
                log(f"❌ Não foi possível inserir '{caminho_arquivo}' com nenhum dos encodings testados.")
                
        else:
            log(f"Formato de arquivo não suportado para '{caminho_arquivo}'. Pulando.")
            continue

In [5]:
def listar_tabelas(conexao):
    """
    Retorna uma lista com o nome de todas as tabelas existentes no banco DuckDB conectado.
    """
    try:
        resultado = conexao.execute(
            "SELECT table_schema, table_name FROM information_schema.tables WHERE table_schema = 'raw' ORDER BY table_name"
        ).fetchall()
        tabelas = [f"{schema}.{nome}" for schema, nome in resultado]
        if not tabelas:
            # Tenta listar todas as tabelas para diagnóstico
            todas = conexao.execute(
                "SELECT table_schema, table_name FROM information_schema.tables ORDER BY table_schema, table_name"
            ).fetchall()
            log(f"Nenhuma tabela encontrada no schema 'raw'. Tabelas disponíveis: {todas}")
        return tabelas
    except Exception as e:
        log(f"Erro ao listar tabelas: {e}")
        return []

def verificar_tabela(conexao, schema, tabela):
    """
    Verifica se uma tabela existe e retorna informações sobre ela.
    """
    try:
        resultado = conexao.execute(
            f"SELECT COUNT(*) FROM {schema}.{tabela}"
        ).fetchone()[0]
        log(f"Tabela {schema}.{tabela} existe com {resultado} linhas.")
        return True
    except Exception as e:
        log(f"Tabela {schema}.{tabela} não existe ou erro ao acessar: {e}")
        return False

## 2: Definição de entradas

In [6]:
# Caminho do arquivo (pode ser CSV, Parquet, etc)
PATH_DIRETORIO = "data_input"

# Caminho do banco DuckDB (arquivo .db)
CAMINHO_DUCKDB = "bd/dev.duckdb"

# Cria a conexão com o banco DuckDB
con = duckdb.connect(CAMINHO_DUCKDB)

In [7]:
inserir_arquivos_no_duckdb(mapear_arquivos_pasta(PATH_DIRETORIO), con)

📁 Schema criado/verificado: raw
[2025-11-20 00:03:16] Inserindo 'data_input\MICRODADOS_CADASTRO_CURSOS_2017.CSV' na tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2017' (tentando encoding: utf-8)...
[2025-11-20 00:03:16] ⚠️ Encoding utf-8 falhou, tentando próximo...
[2025-11-20 00:03:16] Inserindo 'data_input\MICRODADOS_CADASTRO_CURSOS_2017.CSV' na tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2017' (tentando encoding: latin-1)...
[2025-11-20 00:03:18] ✅ Tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2017' criada e dados inseridos com sucesso (encoding: latin-1).
[2025-11-20 00:03:18] Inserindo 'data_input\MICRODADOS_CADASTRO_CURSOS_2018.CSV' na tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2018' (tentando encoding: utf-8)...
[2025-11-20 00:03:18] ⚠️ Encoding utf-8 falhou, tentando próximo...
[2025-11-20 00:03:18] Inserindo 'data_input\MICRODADOS_CADASTRO_CURSOS_2018.CSV' na tabela 'raw.MICRODADOS_CADASTRO_CURSOS_2018' (tentando encoding: latin-1)...
[2025-11-20 00:03:21] ✅ Tabela 'raw.MICRODADOS_CADASTRO_CURSOS_20

In [8]:
listar_tabelas(con)

['raw.MICRODADOS_CADASTRO_CURSOS_2017',
 'raw.MICRODADOS_CADASTRO_CURSOS_2018',
 'raw.MICRODADOS_CADASTRO_CURSOS_2023',
 'raw.MICRODADOS_CADASTRO_CURSOS_2024']

In [9]:
#Criação dos schemas
for schema in  ["staging", "intermediate", "mart"]:
    con.execute(f"CREATE SCHEMA IF NOT EXISTS {schema};")

In [10]:
con.close()